# K-Means


The penguins are back! Use K-Means to find groups from the penguins dataset. Do they correspond to the true labels?

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [2]:
import pandas as pd
import seaborn as sns 
from palmerpenguins import load_penguins
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
sns.set_style('whitegrid')

In [3]:
penguins = load_penguins()
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [4]:
penguins = penguins.dropna()

In [5]:
X = penguins[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
#TODO train a k-means model. What k are you looking for?

sk_kmeans=KMeans(n_clusters=3,random_state=0,n_init=10)
sk_kmeans.fit(X)

KMeans(n_clusters=3, n_init=10, random_state=0)

In [108]:
class KMeans_:
    def __init__(self,n_clusters,random_state=None):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.clusters=None

    def fit(self,X,max_iters=1000,centroids=None):    #adapted from https://neptune.ai/blog/k-means-clustering
        if self.random_state is not None:
            np.random.seed(self.random_state)    #set a random seed if the random state parameter was passed in creation of object

        if centroids is None:
            index_centroids=np.random.choice(X.shape[0],self.n_clusters,replace=False)
            self.centroids=X[index_centroids,:]       #Select random centroids from the data
        else:
            self.centroids=centroids   # centroids are the ones passed in the method

        diff=1
        j=0
        iters=0
        dists=np.zeros((X.shape[0],len(self.centroids)))
        while diff!=0 and iters<max_iters:
            iters+=1
            for i,centroid in enumerate(self.centroids):
                d=centroid-X    #difference between centroid and every other point, shape should be (n-c,4) with n samples and c number of centroids
                dd=d**2    #difference squared  should keep dimensions of d
                d_euc=np.sqrt(np.sum(dd,axis=1))     # compute euclidean distance from centroid to all pointt. Expected shape (n-c,1) or maybe (n-c,)
                dists[:,i]=d_euc   # save distances for each centroid
            
            self.clusters=np.argmin(dists,axis=1)  # get cluster aggrupation by minnimum distances

            new_centroids=np.zeros(self.centroids.shape)
            for i in range(self.n_clusters):
                centroid_indx=np.where(self.clusters==i)   # get indexes to group clusters
                new_centroids[i,:]=np.mean(X[centroid_indx],axis=0)  # compute new centroids with only points of cluster corresponding to centroid

            if j==0:
                diff=1
                j+=1
            else:
                diff=(new_centroids-self.centroids).sum()
            self.centroids=new_centroids

        # print("iters",iters)

    def predict(self):
        pass


In [110]:
jk_kmeans=KMeans_(3,0)
jk_kmeans.fit(X)

In [120]:
sk_kmeans.cluster_centers_,jk_kmeans.centroids

(array([[ 0.65475809, -1.10270671,  1.16246305,  1.10121078],
        [ 0.67202511,  0.80526339, -0.28934658, -0.38410387],
        [-1.04680889,  0.48662566, -0.88169491, -0.7627539 ]]),
 array([[ 0.67202511,  0.80526339, -0.28934658, -0.38410387],
        [-1.04680889,  0.48662566, -0.88169491, -0.7627539 ],
        [ 0.65475809, -1.10270671,  1.16246305,  1.10121078]]))

In [119]:
#Make a plot where the color is the label assigned by the clustering

fig,axs=plt.subplots(1,2,figsize=(8,4),tight_layout=True)

axs[0].scatter(X[:,0],X[:,1],c=sk_kmeans.labels_,cmap="Set1")
axs[0].scatter(sk_kmeans.cluster_centers_[:,0],sk_kmeans.cluster_centers_[:,1],c="k")
axs[0].set_title("KMeans using sklearn library")
axs[0].set_xlabel("bill_length_mm")
axs[0].set_ylabel("bill_depth_mm")

axs[1].scatter(X[:,0],X[:,1],c=jk_kmeans.clusters,cmap="Set1")
axs[1].scatter(jk_kmeans.centroids[:,0],jk_kmeans.centroids[:,1],c="k")
axs[1].set_title("KMeans using handmade implementation")
axs[1].set_xlabel("bill_length_mm")
axs[1].set_ylabel("bill_depth_mm")

plt.show()

In [130]:
#Make a 3D plot where the color is the label assigned by the clustering
fig=plt.figure(figsize=(12,6))
axs=fig.add_subplot(1,2,1,projection='3d')

axs.scatter(X[:,0],X[:,1],X[:,2],c=sk_kmeans.labels_,cmap="Set1")
axs.scatter(sk_kmeans.cluster_centers_[:,0],sk_kmeans.cluster_centers_[:,1],sk_kmeans.cluster_centers_[:,2],c="k")
axs.set_title("KMeans using sklearn library")
axs.set_xlabel("bill_length_mm")
axs.set_ylabel("bill_depth_mm")
axs.set_zlabel("flipper_length_mm")

axs=fig.add_subplot(1,2,2,projection='3d')
axs.scatter(X[:,0],X[:,1],X[:,2],c=jk_kmeans.clusters,cmap="Set1")
axs.scatter(jk_kmeans.centroids[:,0],jk_kmeans.centroids[:,1],jk_kmeans.centroids[:,2],c="k")
axs.set_title("KMeans using handmade implementation")
axs.set_xlabel("bill_length_mm")
axs.set_ylabel("bill_depth_mm")
axs.set_zlabel("flipper_length_mm")

plt.show()

In [ ]:
#TODO To which label each class correspond? Search about scipy linear_sum_assignment
#It is an optimization problem.
#Show the table

In [ ]:
#TODO Do the elbow method for different k's

In [ ]:
#TODO train 1000 models, with random initialization. Plot the results of the losses

In [ ]:
#TODO train 1000 models, with a better initialization. Plot the results of the losses

We have already seen the K-Means class. We know that it can help us to reduce dimensionality and understand how the data behave. We can use it for finding groups and classify them. Now, we are going to use it to do image segmentation.  

What do you think are the data points?  
How can we evaluate each point?  
How is different an image from a normal matrix of data?  

We are going to use RGB images. They have three channels and we must take those into account while computing the groups.

<img src="three_d_array.png" alt="three_d_array" width="300"/>

This time, you will have to implement K-Means by hand, do not use the sklearn. How can you define the centroids?

In [ ]:
#Import libraries
from skimage import io
from skimage import color

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%matplotlib inline
import requests

In [ ]:
#Download image. Feel free to change the image. Remember that we are working with RGB images.
url = "https://www.rainforest-alliance.org/wp-content/uploads/2021/06/poison-dart-frog-thumb-1-400x400.jpg"
img_data = requests.get(url).content
with open('image.jpg', 'wb') as handler:
    handler.write(img_data)
img = io.imread("image.jpg")

In [ ]:
#Show image
plt.imshow(img)
plt.axis("off")

In [ ]:
#Check the shape of the image. If there are more or less channels, we might have an image in another format
img.shape

In [ ]:
#Show just the blue channel
plt.imshow(img[:,:,2])

In [ ]:
#TODO Feel free to change the method definition and documentation, you can create the method that you need

#Method for computing the new centroids. The parameters are:
#     - centroids. Current centroids.
#     - data. data[0] will contain the image
#             data[1] will contain the current label or centroid
# Returns the new centroids. The data and labels do not change.
def compute_centroids(centroids,data):
    #For each centroid, calculate the new center, by finding the mean of the data points in each channel.

    return None

In [ ]:
#TODO Feel free to change the method definition and documentation, you can create the method that you need

#Method for assigning the points to a new centroid. The parameters are:
#     - centroids. Current centroids.
#     - data. data[0] will contain the image
#             data[1] will contain the current label or centroid
# Returns the new labels. The points are assigned based on Euclidean distance.
def assign(centroids,data):

    return None

In [ ]:
#TODO Feel free to change the method definition and documentation, you can create the method that you need

#Method that implements K-Means. The parameters are:
#     - k. The problem with K-Means is that it needs a hyperparameter k, selected by the user.
#     - image. 
#Returns a segmented image.
def k_means(k,image):
    #Generate the centroids. Randomly. Our centroids are three dimensional.
    centroids = {i+1:[np.random.randint(255),np.random.randint(255),np.random.randint(255)]for i in range(k)}
    
    return new_i

In [ ]:
#Get 8 new images, with a different number of clusters
images = []
for k in range(1,9):
    images.append(k_means(k,img))

In [ ]:
#Method for showing the comparison of the images
def print_images(imagenes):
    #Configuration of the plot
    plt.figure(figsize=(15, 10))
    plt.subplot(np.ceil(len(imagenes)/3), 3, 1)

    #Show original image
    plt.imshow(img)
    plt.title("Imagen Original")
    plt.axis("off")

    #Show the segmented images
    for idx in range(len(imagenes)):
        plt.subplot(np.ceil(len(imagenes)/3), 3, 2 + idx)
        plt.imshow(imagenes[idx].astype('uint8'))
        plt.title(f"{idx+1} Colors")
        plt.axis("off")
    plt.show()

In [ ]:
print_images(images)

In [ ]:
plt.imshow(k_means(15,img))